## 1. 데이터 다운로드

https://www.kaggle.com/paultimothymooney/poetry/data 
에서 가사 데이터 받아오기

```
$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기
```

## 2. 데이터 읽어오기

In [102]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:2])

데이터 크기: 187088
Examples:
 ['Look, I was gonna go easy on you and not to hurt your feelings', "But I'm only going to get this one chance"]


# 3. 데이터 정제
 - process_sentecne()함수로 정제하기.
     - 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거. 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가면 잘라내기
         
         
         

In [103]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [104]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> look , i was gonna go easy on you and not to hurt your feelings <end>',
 '<start> but i m only going to get this one chance <end>',
 '<start> something s wrong , i can feel it six minutes , slim shady , you re on <end>',
 '<start> just a feeling i ve got , like something s about to happen , but i don t know what <end>',
 '<start> if that means , what i think it means , we re in trouble , big trouble , <end>',
 '<start> and if he is as bananas as you say , i m not taking any chances <end>',
 '<start> you were just what the doctor ordered i m beginning to feel like a rap god , rap god <end>',
 '<start> all my people from the front to the back nod , back nod <end>',
 '<start> now who thinks their arms are long enough to slap box , slap box ? <end>',
 '<start> they said i rap like a robot , so call me rapbot but for me to rap like a computer must be in my genes <end>']

In [105]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15,padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   4    5   57 ...   21 1079    3]
 [   2   36    5 ...    0    0    0]
 [   4    5   32 ...   54   18    3]
 ...
 [  29   13  139 ... 3354  348    3]
 [   2 6041   29 ...    0    0    0]
 [   8   42   26 ...  348  348    3]] <keras_preprocessing.text.Tokenizer object at 0x7f7dc0f4b190>


In [106]:
print(tensor[:3, :10])

[[   4    5   57   96   53  413   18    7    8   70]
 [   2   36    5   22  124  194   10   43   42   58]
 [   4    5   32  101   11  712 2474    4 1012  866]]


In [107]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [108]:
'''
src_input = tensor[:, :14]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:15는]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.
'''
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(len(src_input[0]))
print(tgt_input[0])
print(len(tgt_input[0]))

[   4    5   57   96   53  413   18    7    8   70   10  441   21 1079]
14
[   5   57   96   53  413   18    7    8   70   10  441   21 1079    3]
14


In [109]:
tokenizer.num_words

12000

# 4. 평가 데이터셋 분리

 - tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!
 
 ```
 enc_train, enc_val, dec_train, dec_val = <코드 작성>
 
 print("Source Train:", enc_train.shape)
 print("Target Train:", dec_train.shape)
 ```
 
output
```

Source Train: (124960, 14)    
Target Train: (124960, 14)
```

In [110]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val =train_test_split(src_input,tgt_input
                                                 ,test_size=0.2, random_state=40)

In [111]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
enc_train

Source Train: (140599, 14)
Target Train: (140599, 14)


array([[ 2, 65, 16, ...,  0,  0,  0],
       [ 2, 31, 24, ...,  0,  0,  0],
       [ 2, 10, 12, ...,  0,  0,  0],
       ...,
       [ 2,  7, 32, ..., 11,  3,  0],
       [ 2,  5, 22, ...,  0,  0,  0],
       [ 2, 68, 55, ...,  0,  0,  0]], dtype=int32)

In [113]:

BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 5. 인공지능 만들기
 - 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)
 
 ```
 loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
 ```
    
 ```
 generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
 ```

In [114]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #origin -256
hidden_size = 1024 #origin - 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

model.summary()


Model: "text_generator_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     multiple                  3072256   
_________________________________________________________________
lstm_22 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_23 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_11 (Dense)             multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [115]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)


model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 66s 121ms/step - loss: 3.6926
Epoch 2/10
549/549 [==============================] - 67s 122ms/step - loss: 3.2022
Epoch 3/10
549/549 [==============================] - 67s 122ms/step - loss: 3.0202
Epoch 4/10
549/549 [==============================] - 69s 126ms/step - loss: 2.8801
Epoch 5/10
549/549 [==============================] - 68s 124ms/step - loss: 2.7613
Epoch 6/10
549/549 [==============================] - 67s 122ms/step - loss: 2.6523
Epoch 7/10
549/549 [==============================] - 71s 130ms/step - loss: 2.5520
Epoch 8/10
549/549 [==============================] - 67s 121ms/step - loss: 2.4570
Epoch 9/10
549/549 [==============================] - 67s 123ms/step - loss: 2.3674
Epoch 10/10
549/549 [==============================] - 70s 127ms/step - loss: 2.2821


In [116]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [117]:
generate_text(model, tokenizer, init_sentence="<start> drunken", max_len=15)

'<start> drunken <unk> , holy <unk> <end> '

In [118]:
generate_text(model, tokenizer, init_sentence="<start> I m", max_len=15)

'<start> i m gonna make you happy <end> '

In [119]:
generate_text(model, tokenizer, init_sentence="<start> piss", max_len=15)

'<start> piss on the floor <end> '

In [120]:
generate_text(model, tokenizer, init_sentence="<start> lyrics", max_len=15)

'<start> lyrics in the club with a <unk> <end> '